In [28]:
# Qiskit and external libraries

import numpy as np
from qiskit.quantum_info import Statevector
from qiskit.quantum_info.operators import Operator
from qiskit.circuit.library import HGate, XGate, CXGate, TGate, PhaseGate

from qiskit import *
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator

import matplotlib.pyplot as plt

In [29]:
# General operations for the state vector |Psi>

# Initial state vector |Psi> as numpy array
# State vector Psi = |01>, change state vector as desired

#psi = np.array([1/4, 1/2, 1/np.sqrt(2), np.sqrt(3)/4], dtype=complex)  # |01> in the computational basis
psi = np.array([0, 1/np.sqrt(2), -1/np.sqrt(2), 0], dtype=complex)

# Quantum gates
H = Operator(HGate()).data  # 2x2 matrix for Hadamard gate # Hadamard (H)
X = Operator(XGate()).data  # 2x2 matrix for Pauli-X gate # Pauli-X (X)

I = np.array([[1, 0], [0, 1]])

# Tensor product
H_tensor_X = np.kron(H, X)

# (H ⊗ X) operator to state vector |Psi>
resulting_state = np.dot(H_tensor_X, psi)

# Results
print("The resulting state vector after applying (H ⊗ X) to |Psi> is:")
print(resulting_state)
print(H_tensor_X)

#CNOT = Operator(CXGate())
#P = Operator(PhaseGate(np.pi)) # Check Theta
#print(CNOT)

The resulting state vector after applying (H ⊗ X) to |Psi> is:
[ 0.5+0.j -0.5+0.j  0.5+0.j  0.5+0.j]
[[ 0.        +0.j  0.70710678+0.j  0.        +0.j  0.70710678+0.j]
 [ 0.70710678+0.j  0.        +0.j  0.70710678+0.j  0.        +0.j]
 [ 0.        +0.j  0.70710678+0.j -0.        +0.j -0.70710678+0.j]
 [ 0.70710678+0.j  0.        +0.j -0.70710678+0.j -0.        +0.j]]


In [ ]:
# Comparisson of built-in Toffoli and decomposed Toffoli
# Exercise 4.23 from "Introduction to Classical and Quantum Computing", Thomas G. Wang (2022)

# Decomposed Toffoli Circuit (Clifford+T decomposition)
def decomposed_toffoli():
    qc = QuantumCircuit(3)

    qc.h(0)
    qc.cx(1, 0)
    qc.tdg(0)
    qc.cx(2, 0)
    qc.t(0)
    qc.cx(1, 0)
    qc.tdg(0)
    qc.cx(2, 0)

    qc.t(0)
    qc.tdg(1)

    qc.t(0)
    qc.cx(1, 0)

    qc.tdg(1)
    qc.cx(1, 0)

    qc.s(1)
    qc.t(2)

    #qc.tdg(2)
    #qc.t(2)
    #qc.cx(0, 2)
    #qc.tdg(2)
    #qc.cx(1, 2)
    #qc.t(2)
    #qc.h(2)
    #qc.tdg(1)
    #qc.cx(0, 1)
    #qc.t(1)
    #qc.cx(0, 1)
    #qc.s(1)
    #qc.t(2)

    return qc

# Built-in Toffoli Circuit
def built_in_toffoli():
    qc = QuantumCircuit(3)
    # Apply the Toffoli (CCX) gate
    qc.ccx(0, 1, 2)

    return qc


# Comparison
def compare_toffoli_circuits():
    # Input state
    input_state = QuantumCircuit(3)
    input_state.x(0)  # Set qubit 0 to |1>
    input_state.x(1)  # Set qubit 1 to |1>

    # Copies of input state to use in both circuits
    qc_decomposed = input_state.compose(decomposed_toffoli())
    qc_builtin = input_state.compose(built_in_toffoli())

    # Measurements of both circuits
    qc_decomposed.measure_all()
    qc_builtin.measure_all()

    # Simulation
    simulator = AerSimulator()

    # Decomposed Toffoli
    result_decomposed = simulator.run(qc_decomposed).result()
    counts_decomposed = result_decomposed.get_counts()

    # Built-in Toffoli
    result_builtin = simulator.run(qc_builtin).result()
    counts_builtin = result_builtin.get_counts()


    # Plotss
    print("Decomposed Toffoli Circuit:")
    print(qc_decomposed)
    qc_decomposed.draw(output = 'mpl')
    plt.show()

    print("Built-in Toffoli Circuit:")
    print(qc_builtin)
    qc_builtin.draw('mpl')
    plt.show()

    print("Measurement Results:")
    plot_histogram([counts_decomposed, counts_builtin],
                   legend=['Decomposed Toffoli', 'Built-in Toffoli'])
    plt.show()


# Run comparison
compare_toffoli_circuits()

Decomposed Toffoli Circuit:
        ┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌───┐       »
   q_0: ┤ X ├┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ T ├┤ X ├───────»
        ├───┤└───┘└─┬─┘└─────┘└─┬─┘└───┘└─┬─┘├─────┤└─┬─┘└───┘└───┘└─┬─┘┌─────┐»
   q_1: ┤ X ├───────■───────────┼─────────■──┤ Tdg ├──┼──────────────■──┤ Tdg ├»
        └───┘                   │            └─────┘  │  ┌───┐          └─────┘»
   q_2: ────────────────────────■─────────────────────■──┤ T ├─────────────────»
                                                         └───┘                 »
meas: 3/═══════════════════════════════════════════════════════════════════════»
                                                                               »
«        ┌───┐      ░ ┌─┐      
«   q_0: ┤ X ├──────░─┤M├──────
«        └─┬─┘┌───┐ ░ └╥┘┌─┐   
«   q_1: ──■──┤ S ├─░──╫─┤M├───
«             └───┘ ░  ║ └╥┘┌─┐
«   q_2: ───────────░──╫──╫─┤M├
«                   ░  ║  ║ └╥┘
«meas: 3/══════════